# ABoffsets_LSSTobs_calspec

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, and others

Created: 2024.11.15

Updated: 2025.01.29 (and it has cool stuff in here...I'm impressed with us)

Updated: 2025.05.07 (generalized for calspec stars beyond just C26202)

## 1. Initial Setup...

### 1.1 Import useful python packages

In [1]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import lsst.geom as geom

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

### 1.2 Include user input

In [2]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
repo = '/repo/dp1'
collections = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241231

plotCutouts = False
verbose = 1         # 0, 1, 2, ...  Larger means more output to the screen.

# Which flux to use?  psfFlux or calibFlux?
fluxName = 'psfFlux'
fluxerrName = 'psfFluxErr'

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '~/Downloads/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '~/Downloads/c26202_mod_008.fits'
               }

# RA, DEC of calspec star in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [3]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [4]:
def cutout_im(butler, ra, dec, datasetType, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a preliminary_visit_image at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    wcs = butler.get('%s.wcs' % datasetType,**dataId)
    xy = geom.PointI(wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(datasetType, parameters=parameters, **dataId)

    return cutout_image

In [5]:
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

In [6]:
def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))]
    frame_one = frames[0]
    frame_one.save("animation.gif", format="GIF", append_images=frames,
               save_all=True, duration=500, loop = 0)

## 2. Calculate Synthetic AB magnitudes for CalSpec star, based on official filter bandpasses

### 2.1 Set up appropriate hardware and system from `syseng_throughputs`

***Check if the following code cell is appropriate for LSSTCam!!!:***

In [7]:
defaultDirs = st.setDefaultDirs()
if instrument == "LSSTComCam":
    #Change detectors from (default) LSST to ComCam (ITL CCDs)
    defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)


### 2.2 Calculate synthetic mags

In [8]:
mags = {}

# Loop through all SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])
    
    

stiswfcnic_007 ~/Downloads/c26202_stiswfcnic_007.fits
mod_008 ~/Downloads/c26202_mod_008.fits


### 2.3 Convert mags numpy arrays into a pandas dataframe

In [9]:
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572800,17.586964
g,16.691931,16.692687
r,16.362017,16.361654
i,16.260196,16.259542
z,16.243679,16.243690
y,16.238847,16.238887


## 3. Query USDF Butler for observations of CalSpec star

### 3.1 Instantiate Butler

In [10]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `preliminary_visit_image`'s that overlap the sky position of CalSpec star

#### 3.2.1 Find the `dataId`'s for all `preliminary_visit_image`'s in this repo/collection that overlap the RA, DEC of CalSpec star

In [11]:
datasetRefs = butler.query_datasets("preliminary_visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)
    if ((verbose < 2) & (i >= 10)): 
        print("...")
        break
    

print(f"\nFound {len(datasetRefs)} preliminary_visit_images")

0 {instrument: 'LSSTComCam', detector: 5, visit: 2024120500122, band: 'r', day_obs: 20241205, physical_filter: 'r_03'}
1 {instrument: 'LSSTComCam', detector: 5, visit: 2024120900310, band: 'r', day_obs: 20241209, physical_filter: 'r_03'}
2 {instrument: 'LSSTComCam', detector: 1, visit: 2024120600085, band: 'i', day_obs: 20241206, physical_filter: 'i_06'}
3 {instrument: 'LSSTComCam', detector: 4, visit: 2024112900293, band: 'g', day_obs: 20241129, physical_filter: 'g_01'}
4 {instrument: 'LSSTComCam', detector: 2, visit: 2024112900212, band: 'r', day_obs: 20241129, physical_filter: 'r_03'}
5 {instrument: 'LSSTComCam', detector: 5, visit: 2024110800251, band: 'r', day_obs: 20241108, physical_filter: 'r_03'}
6 {instrument: 'LSSTComCam', detector: 5, visit: 2024120800369, band: 'z', day_obs: 20241208, physical_filter: 'z_03'}
7 {instrument: 'LSSTComCam', detector: 2, visit: 2024121000418, band: 'r', day_obs: 20241210, physical_filter: 'r_03'}
8 {instrument: 'LSSTComCam', detector: 3, visit:

#### 3.2.2 Plot the cutouts for all these `preliminary_visit_image`'s

***Clean up and eneralize this next section.  For now, grab first r-band image in the list.***

In [12]:
# Find first r-band image from datasetRefs...

for i, ref in enumerate(datasetRefs): 
    if (ref.dataId['band'] == 'r'): 
        break
print(ref.dataId)
visit = ref.dataId['visit']
detector = ref.dataId['detector']

{instrument: 'LSSTComCam', detector: 5, visit: 2024120500122, band: 'r', day_obs: 20241205, physical_filter: 'r_03'}


In [13]:
preliminary_visit_image = butler.get('preliminary_visit_image', dataId={'visit': visit, 'detector': detector})

In [14]:
preliminary_visit_image_info = preliminary_visit_image.getInfo()

In [15]:
visit_info = preliminary_visit_image_info.getVisitInfo()
summary_info = preliminary_visit_image_info.getSummaryStats()

In [16]:
visit_info

VisitInfo(exposureTime=30, darkTime=30.4428, date=2024-12-06T05:37:36.788507079, UT1=nan, ERA=2.78264 rad, boresightRaDec=(52.9777113277, -28.1499767094), boresightAzAlt=(264.9513951398, +58.8809691279), boresightAirmass=1.16772, boresightRotAngle=1.87871 rad, rotType=1, observatory=-30.2446N, -70.7494E  2663, weather=Weather(8.15, 74000, 27.95), instrumentLabel='LSSTComCam', id=2024120500122, focusZ=1.87298, observationType='science', scienceProgram='BLOCK-320', observationReason='science', object='ECDFS', hasSimulatedContent=false)

In [17]:
summary_info

ExposureSummaryStats(version=0, psfSigma=2.233931797333603, psfArea=81.09658383982728, psfIxx=4.6411889986891985, psfIyy=5.371265849248445, psfIxy=-0.1563842672360822, ra=53.0142103872266, dec=-27.917089907482872, pixelScale=0.20033072778283423, zenithDistance=31.330438501529464, expTime=30.0, zeroPoint=32.03103658836926, skyBg=853.252197265625, skyNoise=33.76687132808404, meanVar=1033.0842716804161, raCorners=[52.85526343371607, 52.93312046185906, 53.1730007101663, 53.09550076937295], decCorners=[-27.991266200394474, -27.775321909836432, -27.842673786235085, -28.058732563542563], astromOffsetMean=0.008679303480512036, astromOffsetStd=0.005152637415870528, nPsfStar=93, psfStarDeltaE1Median=-0.00046731811016797846, psfStarDeltaE2Median=0.001615477493032822, psfStarDeltaE1Scatter=0.011891344615878317, psfStarDeltaE2Scatter=0.010499481898178212, psfStarDeltaSizeMedian=0.003058187153306857, psfStarDeltaSizeScatter=0.01624395017489596, psfStarScaledDeltaSizeScatter=0.007342770661889677, psf

In [18]:
datasetType = 'preliminary_visit_image'
dataId = {'visit': visit, 'detector': detector}
preliminary_visit_image = butler.get(datasetType, dataId=dataId)

In [19]:
print(butler.registry.getDatasetType(datasetType))

DatasetType('preliminary_visit_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF)


In [20]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(preliminary_visit_image.image)
    plt.show()

In [21]:
# Create cutout image...

cutoutsize = 501 #Defining the size of the cutout box in pixels
cutout_preliminary_visit_image = cutout_im(butler, raDeg, decDeg, 'preliminary_visit_image', visit, detector, cutoutSideLength=cutoutsize)

In [22]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(cutout_preliminary_visit_image.image)
    plt.show()

#### 3.2.3 Create a pandas Dataframe containing the `recalibrated_star_detector` (formerly `sourceTable`) info for all these `preliminary_visit_image`'s

Now, loop over the `datasetRefs` again, but this time grab the contents of the `recalibrated_star_detector` table for each `ref` and combine into all into one big pandas DataFrame.  

In [23]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('recalibrated_star_detector', dataId=dataId)
    src_list.append(src.to_pandas())

    if ((verbose >= 2) | (i < 10)): 
        print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")
    if ((verbose < 2) & (i == 10)): 
        print("...")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


0 Visit 2024120500122, Detector 5:  Retrieved catalog of 2497 sources.
1 Visit 2024120900310, Detector 5:  Retrieved catalog of 1684 sources.
2 Visit 2024120600085, Detector 1:  Retrieved catalog of 1846 sources.
3 Visit 2024112900293, Detector 4:  Retrieved catalog of 1604 sources.
4 Visit 2024112900212, Detector 2:  Retrieved catalog of 2737 sources.
5 Visit 2024110800251, Detector 5:  Retrieved catalog of 2808 sources.
6 Visit 2024120800369, Detector 5:  Retrieved catalog of 1672 sources.
7 Visit 2024121000418, Detector 2:  Retrieved catalog of 867 sources.
8 Visit 2024111900094, Detector 3:  Retrieved catalog of 850 sources.
9 Visit 2024111700154, Detector 0:  Retrieved catalog of 1489 sources.
...

Total combined catalog contains 1665568 sources.


Let's look at the result:

In [24]:
src_all

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,sourceId
0,52.904754,-27.858450,0,2512.106004,23.119362,0.110007,0.117454,52.904754,-27.858450,5.745434e-06,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146761
1,52.914526,-27.831140,0,3026.617843,22.112920,0.240403,0.206981,52.914526,-27.831140,1.034801e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146762
2,52.915839,-27.828741,0,3074.023404,28.919735,0.100962,0.104703,52.915839,-27.828741,5.135850e-06,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146764
3,52.863247,-27.973842,0,336.379056,25.014509,0.715540,0.731315,52.863247,-27.973842,3.586473e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146765
4,52.863579,-27.973064,0,351.320595,25.791738,0.513893,0.360174,52.863579,-27.973064,1.852596e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665563,53.321904,-27.945975,600452144746202519,244.271520,3938.792488,0.632477,0.639717,53.321904,-27.945975,3.141562e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202927
1665564,53.322421,-27.945850,600452144746202519,243.451779,3947.252249,0.766663,0.605894,53.322421,-27.945850,3.092316e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202928
1665565,53.298363,-27.890035,600452144746202521,1317.177326,3949.596488,0.010865,0.012085,53.298363,-27.890035,5.872180e-07,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202929
1665566,53.299479,-27.889969,600452144746202521,1311.939256,3966.573672,0.401543,0.397382,53.299479,-27.889969,1.957672e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202930


### 3.3 Extract only those rows containing CalSpec star from the src_all catalog

In [25]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
        (~src_all.extendedness_flag) & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# Create SkyCoord object for the coordinates of CalSpec star
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=src_all_cleaned['ra'].values*u.degree, 
                     dec=src_all_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = src_all_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = src_all_cleaned[mask_sep].copy()
nearby_good_df['separation_calspec'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_calspec').drop_duplicates(subset=orig_columns, keep='first')


Add mag_obs and mag_obsErr columns:

In [26]:
# Flux in nano-Janskys to AB magnitudes:
best_df['mag_obs'] = -2.5*np.log10(best_df[fluxName]) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['mag_obsErr'] = 2.5/math.log(10)*best_df[fluxerrName]/best_df[fluxName]

Display `visit`, `detector`, `band`, fluxName, fluxerrName, `mag_obs`, `mag_obsErr`, and `separation_calspec` from best_df, sorted by `visit` and `band`:

In [27]:
# Set pandas to show all rows...
if verbose > 2:
    pd.set_option("display.max_rows", None)

In [28]:
best_df[['visit', 'detector', 'band', fluxName, fluxerrName, 'mag_obs', 'mag_obsErr', 'separation_calspec']].sort_values(['visit', 'band'])

,visit,detector,band,psfFlux,psfFluxErr,mag_obs,mag_obsErr,separation_calspec
937066,2024110800247,5,r,1.032999e+06,1036.181030,16.364750,0.001089,0.618540
548265,2024110800248,5,i,1.135542e+06,1207.354248,16.261993,0.001154,0.612946
275523,2024110800249,5,i,1.140589e+06,1189.100586,16.257175,0.001132,0.632081
1407743,2024110800250,5,r,1.044996e+06,1025.728516,16.352213,0.001066,0.621688
12178,2024110800251,5,r,1.037818e+06,1040.408691,16.359695,0.001088,0.617553
...,...,...,...,...,...,...,...,...
193962,2024121000427,2,g,7.614986e+05,1489.196167,16.695827,0.002123,0.600141
1523646,2024121000428,5,g,7.611666e+05,1144.746216,16.696301,0.001633,0.607936
1314643,2024121000430,2,g,7.530312e+05,1398.115967,16.707968,0.002016,0.609537
1091420,2024121000433,2,g,7.492108e+05,1759.515137,16.713490,0.002550,0.596536


In [29]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

Number of rows:  594


In [30]:
# Reset pandas to its default maximum rows to print to screen
if verbose > 2:
    pd.reset_option("display.max_rows")

***Do we need to do any further masking/culling in the above table before proceeding?***

## 4. Measure differences between the calibrated observed magnitudes and the LSST Synthetic Mags for CalSpec star

***Some old material that probably can be removed:***

In [31]:
## Reset the index to turn the keys into a column
#df_mags_reset = df_mags.reset_index()
#
## Merge the dataframes based on the filter name
#combined_df = pd.merge(best_df, df_mags_reset, left_on='band', right_on='index')
#
#combined_df

In [32]:
#print(df_mags)

In [33]:
## Group by the 'band' column and calculate the median of 'mag_obs' for each group
#median_values = combined_df.groupby('band')['mag_obs'].median().reset_index()
#median_values = median_values.rename(columns={'mag_obs': 'median_mag_obs'})
#
## Merge the median values back into the combined_df dataframe
#combined_df = pd.merge(combined_df, median_values, on='band', how='left')
#combined_df

In [34]:
## Calculate the number of rows for each filter band
#row_counts = combined_df.groupby('band').size().reset_index(name='n_total')
#
## Merge the row counts back into the combined_df dataframe
#combined_df = pd.merge(combined_df, row_counts, on='band', how='left')
#
#combined_df

In [35]:
## Calculate the differences and add the new columns
#combined_df['offset_stis'] = combined_df['median_mag_obs'] - combined_df['stiswfcnic_007']
#combined_df['offset_mod'] = combined_df['median_mag_obs'] - combined_df['mod_008']
#
#combined_df

**trying again - this time calculate the median then combine the tables for stis and mod**


In [36]:
## Calculate the number of rows for each filter band
#row_counts = best_df.groupby('band').size().reset_index(name='n_band')
#
## Merge the row counts back into the combined_df dataframe
#combined_df = pd.merge(best_df, row_counts, on='band', how='left')
#
#combined_df

In [37]:
# Group by the 'band' column in best_df calculate the counts of 'band' for each group
count_df = best_df.groupby('band')['mag_obs'].count().reset_index()

# Rename the columns for clarity
count_df = count_df.rename(columns={'mag_obs': 'n_band'})

if verbose > 2:
    count_df

In [38]:
# Group by the 'band' column in beset_df and calculate the median of 'mag_obs' for each group
median_df = best_df.groupby('band')['mag_obs'].median().reset_index()

# Rename the columns for clarity
median_df = median_df.rename(columns={'mag_obs': 'median_mag_obs'})

if verbose > 2:
    median_df

In [39]:
# Merge the count_df and merge_df dataframes based on the filter band name
combined_df = pd.merge(count_df, median_df, left_on='band', right_on='band')

if verbose > 2:
    combined_df

In [40]:
# Reset the df_mags index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(combined_df, df_mags_reset, left_on='band', right_on='index')

if verbose > 2:
    combined_df

In [41]:
# Calculate the differences and add the new columns
#combined_df['offset_stis'] = combined_df['median_mag_obs'] - combined_df['stiswfcnic_007']
#combined_df['offset_mod'] = combined_df['median_mag_obs'] - combined_df['mod_008']

for sed_key in sedfile_dict:
    offset_name = """offset_%s""" % (sed_key)
    combined_df[offset_name] = combined_df['median_mag_obs'] - combined_df[sed_key]


if verbose > 2:
    combined_df

In [42]:
# Output final cleaned-up results...

# Define the desired order of 'band'
order = ['u', 'g', 'r', 'i', 'z', 'y']

# Remove the 'index' column
combined_df = combined_df.drop(columns=['index'])

# Reorder the dataframe based on the 'band' column
combined_df['band'] = pd.Categorical(combined_df['band'], categories=order, ordered=True)
combined_df = combined_df.sort_values('band').reset_index(drop=True)

combined_df

,band,n_band,median_mag_obs,stiswfcnic_007,mod_008,offset_stiswfcnic_007,offset_mod_008
0,u,25,17.622753,17.572800,17.586964,0.049953,0.035789
1,g,143,16.699482,16.691931,16.692687,0.007551,0.006795
2,r,170,16.363060,16.362017,16.361654,0.001043,0.001406
3,i,115,16.259958,16.260196,16.259542,-0.000237,0.000416
4,z,116,16.243576,16.243679,16.243690,-0.000102,-0.000113
5,y,24,16.245932,16.238847,16.238887,0.007084,0.007045


**Let's stop here for now:**

In [43]:
raise StopExecution

## 5. Sandbox